In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import optuna
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as mso
import seaborn as sns
import warnings
import os
import scipy

from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

In [3]:
data_path = '/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv'
cc = pd.read_csv(data_path)
cc

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,647,2011,70%,Austria,3.75,,Peru
1791,Zotter,Congo,749,2011,65%,Austria,3.00,Forastero,Congo
1792,Zotter,Kerala State,749,2011,65%,Austria,3.50,Forastero,India
1793,Zotter,Kerala State,781,2011,62%,Austria,3.25,,India


In [4]:
import re
for column in cc.columns:
    cc.columns = cc.columns.str.replace(r'\s*\n\s*', ' ', regex=True)
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Company (Maker-if known)          1795 non-null   object 
 1   Specific Bean Origin or Bar Name  1795 non-null   object 
 2   REF                               1795 non-null   int64  
 3   Review Date                       1795 non-null   int64  
 4   Cocoa Percent                     1795 non-null   object 
 5   Company Location                  1795 non-null   object 
 6   Rating                            1795 non-null   float64
 7   Bean Type                         1794 non-null   object 
 8   Broad Bean Origin                 1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [5]:
cc.rename(columns={
    'Company (Maker-if known)': 'Company',
    'Specific Bean Origin or Bar Name': 'BarName',
    'Review Date':'ReviewDate',
    'Cocoa Percent':'CocoaPercent',
    'Company Location':'CompanyLocation',
    'Bean Type':'BeanType',
    'Broad Bean Origin' : 'BroadBeanOrigin',
}, inplace=True)

In [6]:
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1795 non-null   object 
 1   BarName          1795 non-null   object 
 2   REF              1795 non-null   int64  
 3   ReviewDate       1795 non-null   int64  
 4   CocoaPercent     1795 non-null   object 
 5   CompanyLocation  1795 non-null   object 
 6   Rating           1795 non-null   float64
 7   BeanType         1794 non-null   object 
 8   BroadBeanOrigin  1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [7]:
cc.isnull().sum()

Company            0
BarName            0
REF                0
ReviewDate         0
CocoaPercent       0
CompanyLocation    0
Rating             0
BeanType           1
BroadBeanOrigin    1
dtype: int64

In [8]:
cc['CocoaPercent'] = cc['CocoaPercent'].str.replace('%', '').astype(float)
cc['CocoaPercent'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1795 entries, 0 to 1794
Series name: CocoaPercent
Non-Null Count  Dtype  
--------------  -----  
1795 non-null   float64
dtypes: float64(1)
memory usage: 14.2 KB


In [9]:
cc['BeanType'] = (cc['BeanType']
                 .str.strip()  # Menghapus leading/trailing spaces
                 .replace('', np.nan))  # Mengganti string kosong dengan NaN
cc['BeanType'] = cc['BeanType'].replace(np.nan, cc['BeanType'].mode()[0])

In [10]:
cc['ReviewDate'] = pd.to_datetime(cc['ReviewDate'].astype(str), format='%Y', errors='coerce')
cc['ReviewDate'] = cc['ReviewDate'].dt.year

In [11]:
cc.isnull().sum()
cc['BroadBeanOrigin'] = cc['BroadBeanOrigin'].fillna(cc['BroadBeanOrigin'].mode()[0])

In [12]:
cc_numerik = cc.select_dtypes(include=np.number)
cc_numerik.info()
cc_object = cc.select_dtypes(include='object')
cc_object.info()

cc = pd.get_dummies(cc, columns=['Company','BarName','CompanyLocation','BeanType'], drop_first=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   REF           1795 non-null   int64  
 1   ReviewDate    1795 non-null   int32  
 2   CocoaPercent  1795 non-null   float64
 3   Rating        1795 non-null   float64
dtypes: float64(2), int32(1), int64(1)
memory usage: 49.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1795 non-null   object
 1   BarName          1795 non-null   object
 2   CompanyLocation  1795 non-null   object
 3   BeanType         1795 non-null   object
 4   BroadBeanOrigin  1795 non-null   object
dtypes: object(5)
memory usage: 70.2+ KB


# Modeling

In [13]:
X = cc.drop(columns='BroadBeanOrigin')
y = cc['BroadBeanOrigin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
LRclassifier = LogisticRegression(solver='saga', random_state=42)
LRclassifier.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(random_state=42, solver='saga')

In [15]:
y_pred = LRclassifier.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

from sklearn.metrics import accuracy_score
LRAcc = accuracy_score(y_pred,y_test)
print('LR accuracy: {:.2f}%'.format(LRAcc*100))

                               precision    recall  f1-score   support

    Africa, Carribean, C. Am.       0.00      0.00      0.00         1
                    Australia       0.00      0.00      0.00         1
                       Belize       0.00      0.00      0.00         9
                      Bolivia       0.00      0.00      0.00        13
                       Brazil       0.00      0.00      0.00        12
                     Cameroon       0.00      0.00      0.00         1
                    Carribean       0.00      0.00      0.00         1
       Central and S. America       0.00      0.00      0.00         1
                     Colombia       0.00      0.00      0.00         6
                        Congo       0.00      0.00      0.00         3
                   Costa Rica       0.00      0.00      0.00         9
            Domincan Republic       0.00      0.00      0.00         6
           Dominican Republic       0.00      0.00      0.00        31
     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
def f1_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)

    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1

In [17]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 10, 200)
#     max_depth = trial.suggest_int('max_depth', 3, 20)
#     min_samples_split= trial.suggest_int('min_samples_split', 2, 20)
    
#     model = RandomForestClassifier(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         random_state=42,
#         bootstrap=True,
#         min_samples_split=min_samples_split,
#         max_features='sqrt'
#     )
    
#     model.fit(X_train, y_train)

#     y_pred = model.predict(X_test)

#     score = f1_score(y_test, y_pred)

#     return score

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials = 20)

# best_params = study.best_params
# print("Best Parameter:", best_params)

# best_model = RandomForestClassifier(
#     n_estimators = best_params['n_estimators'],
#     max_depth = best_params['max_depth'], 
#     random_state=42,
#     bootstrap=True,
#     min_samples_split=5,
#     max_features='sqrt',    
# )

# best_model.fit(X_train, y_train)

# y_pred = best_model.predict(X_test)

# final_f1 = f1_score(y_test,y_pred)
# print("Final f1:", final_f1)